Linear Regression From SciKit-Learn

In [1]:
import numpy as np
from numpy import genfromtxt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math 
from sklearn.preprocessing import PolynomialFeatures

# Load data into a Pandas DataFrame
x_data = genfromtxt('../data/embs/old/model1-10000/layer6/embslayer6-addedfromprevlayer.csv',delimiter=',',encoding='utf-8-sig',skip_header=1)

# Split data into X and y
X = x_data[:,0:128]
#128 --> pka, NMR 
#137  --> ae 
#138 onwards --> electron density 
y = x_data[:,137]

#label to split as well
label = x_data[:,129:133]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, label_train, label_test = train_test_split(X, y, label, test_size=0.2, random_state=50)

use_train_for_test = False

save_pred_filepath = '../data/embs/old/model1-10000/layer6/lrtrainpredictionsAE' #NO .csv!!

In [2]:
# Transform features to quadratic form
#poly = PolynomialFeatures(degree=2)  # Use degree 2 for quadratic regression
#X_train = poly.fit_transform(X_train)
#X_test = poly.fit_transform(X_test)

# Create a LinearRegression object
lr = LinearRegression()

# Standardize the data
#scaler_X = StandardScaler()
#scaler_y = StandardScaler()

#X_train_scaled = scaler_X.fit_transform(X_train)
#X_test_scaled = scaler_X.transform(X_test)

#y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1,1))
#y_test_scaled = scaler_y.transform(y_test.reshape(-1,1))

# Fit the model
lr.fit(X_train, y_train)

if use_train_for_test == True:
    # Make predictions
    predictions = lr.predict(X_train)
#    predictions = scaler_y.inverse_transform(predictions)

    # Evaluate the model
    mse = mean_squared_error(y_train, predictions)
    r2 = r2_score(y_train, predictions)
else:
    # Make predictions
    predictions = lr.predict(X_test)
#    predictions = scaler_y.inverse_transform(predictions_scaled)

    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

print('Mean square-root error:', math.sqrt(mse))
print('R-squared:', r2)


Mean square-root error: 42.772435694030484
R-squared: 0.9960998794004171


In [ ]:
testfile_path = '../data/perturbation/targetall-useavg-subavg/qm9-0-1000-all/d1-p1.csv'
test_data = genfromtxt(testfile_path,delimiter=',',encoding='utf-8-sig',skip_header=1)

# Split data into X and y
X_test = test_data[:,0:128]
#128 --> pka, NMR 
#138 onwards --> electron density 
y_test = test_data[:,128]

predictions = lr.predict(X_test)
#    predictions = scaler_y.inverse_transform(predictions_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)




In [ ]:
print('Mean square-root error:', math.sqrt(mse))
print('R-squared:', r2)

In [ ]:
unique_fgs = np.unique(x_data[:,129])
print(unique_fgs)

#Pearson coefficients on each unique FG

In [ ]:
from scipy.stats import pearsonr

# Calculate the number of features (columns) in X
num_features = X_train.shape[1]

# Initialize an array to store the correlation coefficients
correlated_features_for_all_fgs = []
correlations_of_features_for_all_fgs = []

for each_fg in unique_fgs:
    
    X_fg = x_data[np.where(x_data[:,129] == each_fg)]

    X_data_fg = X_fg[:,0:128]
    y_data_fg = X_fg[:,128]

    correlated_features = []
    correlations_of_features = []

    for each_feature in range(num_features):
        
        feature = X_data_fg[:, each_feature]  # Extract the i-th feature from X

        corr_coeff, p_value = pearsonr(X_data_fg[:,each_feature], y_data_fg)  # Compute Pearson correlation coefficient

        if p_value < 0.005:
            correlated_features.append(each_feature)
            correlations_of_features.append(corr_coeff)
        
    correlated_features_for_all_fgs.append(correlated_features+ [each_fg])

    correlations_of_features_for_all_fgs.append(correlations_of_features + [each_fg])

In [ ]:
print(correlated_features_for_all_fgs)

In [ ]:
import csv
# Specify the CSV file path
csv_file_path = "../data/embs/embspKA-stef/pearsonanalysis/correlated_features.csv"

# Write the uneven list of lists to a CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(correlated_features_for_all_fgs)

print(f"CSV file saved to {csv_file_path}")

In [ ]:
from scipy.stats import pearsonr

# Calculate the number of features (columns) in X
num_features = X_train.shape[1]

# Initialize an array to store the correlation coefficients
strong_correlations = []

# Loop through each feature in X and calculate the Pearson correlation coefficient with y
for i in range(num_features):
    feature = X_train[:, i]  # Extract the i-th feature from X
    corr_coeff, p_value = pearsonr(feature, y_train)  # Compute Pearson correlation coefficient

    # Interpret the results
    if p_value < 0.00005:  # You can choose a significance level (e.g., 0.05)
        correlation_coeffs.append()


In [ ]:

# Print the results
print(f"Pearson Correlation Coefficients: {correlation_coeffs}")
print(f"P-value: {p_values}")

negative_corr = []
positive_corr = []
no_corr = []
for each_feature in range(128):
    # Interpret the results
    if p_values[each_feature] < 0.00005:  # You can choose a significance level (e.g., 0.05)
        if correlation_coeffs[each_feature] > 0:
            positive_corr.append(each_feature)
        elif correlation_coeffs[each_feature] < 0:
            negative_corr.append(each_feature)
        else:
            no_corr.append(each_feature)
    else:
        no_corr.append(each_feature)

print("There is a statistically significant positive correlation: ",positive_corr)
print("There is a statistically significant negative correlation: ",negative_corr)
print("There is no statistically significant correlation:", no_corr)

Parsing predictions of LR by FG

In [ ]:

print(np.shape(label_train))
print(np.shape(predictions))

print(predictions)

for i in range(np.shape(predictions)[1]):

    if use_train_for_test == True:
        y_train = y_train.reshape(-1,1) 
        save_predictions = np.column_stack((predictions[:,i],y_train[:,i],label_train))
    elif use_train_for_test == False: 
        y_test = y_test.reshape(-1,1) 
        save_predictions = np.column_stack((predictions[:,i],y_test[:,i],label_test))
        
    filepath =  save_pred_filepath + '%s.csv' %(i) 
    np.savetxt(filepath,save_predictions,delimiter=',')


Custom testing

In [ ]:
custom_test_path = '../data/embs/embspKA-stef/inductive/embs.csv'

custom_test_data = genfromtxt(custom_test_path,delimiter=',',encoding='utf-8-sig',skip_header=1)

X_tst= custom_test_data[:,0:128]

X_tst_scaled = scaler_X.transform(X_tst)

# Make predictions
predictions_scaled = lr.predict(X_tst_scaled)
predictions = scaler_y.inverse_transform(predictions_scaled)
print(predictions)


In [ ]:
from scipy.stats import t

def pearson_coeff_totaldata(feature,target,apply_data,total_data):

    feature = feature.astype(np.int32)

    x_mean = np.mean(total_data[:,feature])
    y_mean = np.mean(total_data[:,target])

    print(x_mean,y_mean)
    print(apply_data[0][feature])

    correlation_sum_numerator = 0
    correlation_sum_denominator = 0
    for each_data in range(len(apply_data)):

        correlation_sum_numerator = correlation_sum_numerator + (apply_data[each_data][feature]-x_mean)*(apply_data[each_data][target]-y_mean)
        correlation_sum_denominator = correlation_sum_denominator + ((math.sqrt((apply_data[each_data][feature]-x_mean)**2))*(math.sqrt((apply_data[each_data][target]-y_mean)**2)))
    

    pearson = correlation_sum_numerator/correlation_sum_denominator
    df = len(apply_data[:,target]) - 2

    # Calculate t-statistic
    t_statistic = pearson * np.sqrt(df / (1 - pearson ** 2))
    
    # Calculate two-tailed p-value
    p_value = 2 * (1 - t.cdf(abs(t_statistic), df))

    return pearson, p_value

